In [1]:
!pip install audiomentations
!pip install timm
!pip install torchlibrosa

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 244 kB 4.3 MB/s 
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
#import cv2
#import audioread
import logging
import os
import random
import time
import warnings
warnings.simplefilter('ignore')

from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import os,math,random

from contextlib import contextmanager
from typing import Optional, List
from fastprogress import progress_bar

import librosa
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.utils.data as data
import torchvision.models as models

from pytorch_lightning.core import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import label_ranking_average_precision_score

import audiomentations as aaug

from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation

In [3]:
def get_logger(out_file=None):
    logger = logging.getLogger()
    formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    logger.handlers = []
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    handler.setLevel(logging.INFO)
    logger.addHandler(handler)

    if out_file is not None:
        fh = logging.FileHandler(out_file)
        fh.setFormatter(formatter)
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)
    logger.info("logger set up")
    return logger
    
    
@contextmanager
def timer(name: str, logger: Optional[logging.Logger] = None):
    t0 = time.time()
    msg = f"[{name}] start"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)
    yield

    msg = f"[{name}] done in {time.time() - t0:.2f} s"
    if logger is None:
        print(msg)
    else:
        logger.info(msg)

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
set_seed(1213)

In [5]:
PERIOD = 10
EPOCH = 40
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 18
sample_rate = 48000
window_size = 2048
hop_size = 1024
mel_bins = 128
fmax = sample_rate/2.0
fmin = 40
classes_num = 24
is_inference = True  # if True, pls update weight for predict blow

data_dir = '../input/rfcx-species-audio-detection'
train_tp_df = pd.read_csv(os.path.join(data_dir, 'train_tp.csv'))
train_fp_df = pd.read_csv(os.path.join(data_dir, 'train_fp.csv'))
data_test = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

OUTPUT_DIR = './'

## Dataset

In [6]:
class MelSpecDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        df=None,
        num_classes=24,
        effective_length=sample_rate * PERIOD,
        waveform_transforms=None,
        is_train=False,
        data_dir=None
    ):
        self.data_dir = data_dir
        self.is_train = is_train
        self.num_classes = num_classes
        self.df = df
        self.effective_length = effective_length
        self.waveform_transforms = waveform_transforms
        

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        recording_id = self.df.recording_id.values[idx]

        prefix = 'train' if self.is_train else 'test'

        y, sr = librosa.load(
            os.path.join(self.data_dir, prefix , recording_id + '.flac'),
            sr=None, mono=True, res_type="kaiser_fast"
        )

        len_y = len(y)

        # y = butter_bandpass_filter(y, f_min, f_max, order=9)          
        t_min = float(self.df.t_min.values[idx]) * sr
        t_max = float(self.df.t_max.values[idx]) * sr
        
        # Positioning sound slice
        center = np.round((t_min + t_max) / 2)
        beginning = center - self.effective_length / 2
        if beginning < 0:
            beginning = 0
        beginning = np.random.randint( beginning , center)
        ending = beginning + self.effective_length
        if ending > len_y:
            ending = len_y
        beginning = ending - self.effective_length

        y = y[int(beginning):int(ending)].astype(np.float32)

        # Do the augmentations
        if self.waveform_transforms:
            y = self.waveform_transforms(y, sr)
        if self.is_train:
            target = np.zeros(self.num_classes, dtype=np.single)
            target[self.df.species_id.values[idx]] = 1
            return  y, target
        else:
            return  y


train_full_df = train_tp_df

for index, row in train_full_df.iterrows():
    if fmin > float(row['f_min']):
        fmin = float(row['f_min'])
    if fmax < float(row['f_max']):
        fmax = float(row['f_max'])

# Get some safety margin
fmin = int(fmin * 0.9)
fmax = int(fmax * 1.1)



In [7]:
def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)

        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))

In [8]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        # get the number of the inputs
        n = model.in_features
        y = 1.0/np.sqrt(n)
        model.weight.data.normal_(-y, y)
        #model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        # get the number of the inputs
        n = model.in_features
        y = 1.0/np.sqrt(n)
        model.weight.data.normal_(-y, y)
        model.bias.data.zero_()


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x

class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear"):
        super().__init__()

        self.activation = activation
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.tanh(self.att(x)), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)


### Panns Model

## Efficient Net

In [9]:
import timm

In [10]:
class PANNsEfficientNet(nn.Module):
    def __init__(self, sample_rate: int, window_size: int, hop_size: int,
                 mel_bins: int, fmin: int, fmax: int, classes_num: int,top_db=None):
        super().__init__()
        
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        self.interpolate_ratio = 30 # Downsampled ratio
        #self.apply_aug = apply_aug

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(
            n_fft=window_size,
            hop_length=hop_size,
            win_length=window_size,
            window=window,
            center=center,
            pad_mode=pad_mode,
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(
            sr=sample_rate,
            n_fft=window_size,
            n_mels=mel_bins,
            fmin=fmin,
            fmax=fmax,
            ref=ref,
            amin=amin,
            top_db=top_db,
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=64,
            time_stripes_num=2,
            freq_drop_width=8,
            freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_bins)

        self.base_model = timm.create_model('tf_efficientnet_b5_ns', pretrained=True)
        #self.model.head.fc = nn.Linear(n_features, n_class)
        #self.base_model = EfficientNet.from_pretrained('efficientnet-b4')
         
        #layers = list(base_model.children())[:-2]
        #self.encoder = nn.Sequential(*layers

        in_features = self.base_model.classifier.in_features
        
        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlock(in_features, classes_num, activation='sigmoid')
        self.init_weights()

        
    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    
    def preprocess(self, input_x, mixup_lambda=None):

        x = self.spectrogram_extractor(input_x)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        

        
        x = x.transpose(1, 3)
        
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)
        
        # Mixup on spectrogram
        if self.training  and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        return x, frames_num
        

    def forward(self, input_x):
        
        input_x, mixup_lambda = input_x,None
        """
        Input: (batch_size, data_length)"""
        
        x, frames_num = self.preprocess(input_x, mixup_lambda=mixup_lambda)
        
        
        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        
        x = self.base_model.forward_features(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.4, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.4, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "framewise_logit":framewise_logit,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict

## Resnet

In [11]:
class PANNsResNet50Att(nn.Module):
    def __init__(self, sample_rate: int, window_size: int, hop_size: int,
                 mel_bins: int, fmin: int, fmax: int, classes_num: int,top_db=None):
        super().__init__()
        
        window = 'hann'
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        amin = 1e-10
        self.interpolate_ratio = 30  # Downsampled ratio
        #self.apply_aug = apply_aug

        # Spectrogram extractor
        self.spectrogram_extractor = Spectrogram(
            n_fft=window_size,
            hop_length=hop_size,
            win_length=window_size,
            window=window,
            center=center,
            pad_mode=pad_mode,
            freeze_parameters=True)

        # Logmel feature extractor
        self.logmel_extractor = LogmelFilterBank(
            sr=sample_rate,
            n_fft=window_size,
            n_mels=mel_bins,
            fmin=fmin,
            fmax=fmax,
            ref=ref,
            amin=amin,
            top_db=top_db,
            freeze_parameters=True)

        # Spec augmenter
        self.spec_augmenter = SpecAugmentation(
            time_drop_width=32,
            time_stripes_num=2,
            freq_drop_width=4,
            freq_stripes_num=2)

        self.bn0 = nn.BatchNorm2d(mel_bins)

        base_model = torch.hub.load("zhanghang1989/ResNeSt",
                                    'resnest50',
                                    pretrained=True)
        layers = list(base_model.children())[:-2]
        
        self.encoder = nn.Sequential(*layers)

        in_features = base_model.fc.in_features
        self.fc1 = nn.Linear(in_features, in_features, bias=True)
        self.att_block = AttBlock(in_features, classes_num, activation='sigmoid')
        self.init_weights()

        
    def init_weights(self):
        init_bn(self.bn0)
        init_layer(self.fc1)
        
    
    def preprocess(self, input_x, mixup_lambda=None):

        x = self.spectrogram_extractor(input_x)  # (batch_size, 1, time_steps, freq_bins)
        x = self.logmel_extractor(x)  # (batch_size, 1, time_steps, mel_bins)

        frames_num = x.shape[2]
        

        
        x = x.transpose(1, 3)
        
        x = self.bn0(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)
        
        # Mixup on spectrogram
        if self.training  and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        return x, frames_num
        

    def forward(self, input_x):
        input_x, mixup_lambda = input_x,None
        """
        Input: (batch_size, data_length)"""
        #b, c, s = input_x.shape
        
        #input_x = input_x.reshape(b*c, s)
        x, frames_num = self.preprocess(input_x, mixup_lambda=mixup_lambda)
        
        
        # Output shape (batch size, channels, time, frequency)
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3])
        x = self.encoder(x)
        
        # Aggregate in frequency axis
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc1(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_logit = self.att_block.cla(x).transpose(1, 2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        # Get framewise output
        framewise_output = interpolate(segmentwise_output,
                                       self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        framewise_logit = interpolate(segmentwise_logit, self.interpolate_ratio)
        framewise_logit = pad_framewise_output(framewise_logit, frames_num)

        output_dict = {
            "framewise_output": framewise_output,
            "framewise_logit":framewise_logit,
            "logit": logit,
            "clipwise_output": clipwise_output
        }

        return output_dict

In [12]:
def _one_sample_positive_class_precisions(scores, truth):
    num_classes = scores.shape[0]
    pos_class_indices = np.flatnonzero(truth > 0)

    if not len(pos_class_indices):
        return pos_class_indices, np.zeros(0)

    retrieved_classes = np.argsort(scores)[::-1]

    class_rankings = np.zeros(num_classes, dtype=np.int)
    class_rankings[retrieved_classes] = range(num_classes)

    retrieved_class_true = np.zeros(num_classes, dtype=np.bool)
    retrieved_class_true[class_rankings[pos_class_indices]] = True

    retrieved_cumulative_hits = np.cumsum(retrieved_class_true)

    precision_at_hits = retrieved_cumulative_hits[class_rankings[pos_class_indices]] / (
        1 + class_rankings[pos_class_indices].astype(np.float)
    )
    return pos_class_indices, precision_at_hits


def lwlrap(truth, scores):
    assert truth.shape == scores.shape
    num_samples, num_classes = scores.shape
    precisions_for_samples_by_classes = np.zeros((num_samples, num_classes))
    for sample_num in range(num_samples):
        pos_class_indices, precision_at_hits = _one_sample_positive_class_precisions(
            scores[sample_num, :], truth[sample_num, :]
        )
        precisions_for_samples_by_classes[
            sample_num, pos_class_indices
        ] = precision_at_hits

    labels_per_class = np.sum(truth > 0, axis=0)
    weight_per_class = labels_per_class / float(np.sum(labels_per_class))

    per_class_lwlrap = np.sum(precisions_for_samples_by_classes, axis=0) / np.maximum(
        1, labels_per_class
    )
    return per_class_lwlrap, weight_per_class

class ImprovedPANNsLoss(nn.Module):
    def __init__(self, output_key="logit", weights=[1, 0.5]):
        super().__init__()

        self.output_key = output_key
        if output_key == "logit":
            self.normal_loss = nn.BCEWithLogitsLoss()
        else:
            self.normal_loss = nn.BCELoss()

        self.bce = nn.BCELoss()
        self.weights = weights

    def forward(self, input, target):
        input_ = input[self.output_key]
        target = target.float()

        framewise_output = input["framewise_output"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        normal_loss = self.normal_loss(input_, target)
        auxiliary_loss = self.bce(clipwise_output_with_max, target)

        return self.weights[0] * normal_loss + self.weights[1] * auxiliary_loss

In [13]:
class LightModel(LightningModule):
    def __init__(self, model,fold_id = 0, data_dir=data_dir,
                 batch_size=batch_size, num_workers=4, DEVICE=DEVICE, mixup = False):
        super().__init__()
        self.model = model
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.DEVICE = DEVICE
        self.loss_fn = ImprovedPANNsLoss()
        #self.loss_fn = ImprovedFocalLoss()
        self.fold_id = fold_id
        self.mixup = mixup
        #self.train_df, self.val_df = train_df, valid_df

    def forward(self, batch):
        return self.model(batch)
    
    def training_step(self, batch, batch_idx):
        y, target = [x.to(self.DEVICE) for x in batch]
        
        if self.mixup:
            mixup_augmenter = Mixup(1.)
            mix_y = mixup_augmenter.get_lambda(batch_size=batch_size, device = DEVICE)
            mix_target = do_mixup(target,mixup_augmenter.get_lambda(batch_size=batch_size, device = DEVICE))
            output = model((y,mix_y))
            bceLoss = self.loss_fn(output, mix_target)
        else:
            output = model((y,None))
            bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        return loss
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        logs = {"train_loss": avg_loss}
    
    def validation_step(self, batch, batch_idx):
        y, target = [x.to(self.DEVICE) for x in batch]
        output = model((y,None))
        bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        p = torch.sigmoid(output['clipwise_output'])
        score_class, weight = lwlrap(target.cpu().numpy(), p.cpu().numpy())
        score = (score_class * weight).sum()
        self.log("Lwlrap_epoch", score, on_epoch=True, prog_bar=True)

    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, amsgrad = True, weight_decay = 0.01)
        #optimizer = AdamP(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10,)
        lr_scheduler = {"scheduler": scheduler }
        return [optimizer], [lr_scheduler]
    
    
    def prepare_data(self):
        
        image_source = train_tp_df[['recording_id', 'species_id']].drop_duplicates()

        # get lists for image_ids and sources
        image_ids = image_source['recording_id'].to_numpy()
        sources = image_source['species_id'].to_numpy()

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        split = skf.split(image_ids, sources) # second arguement is what we are stratifying by

        select = self.fold_id #Use fold
        train_ix, val_ix = next(islice(split, select, select+1))

            # translate indices to ids
        train_ids = image_ids[train_ix]
        val_ids = image_ids[val_ix]

        # create corresponding dfs
        self.train_df = train_tp_df[train_tp_df['recording_id'].isin(train_ids)]
        self.val_df = train_tp_df[train_tp_df['recording_id'].isin(val_ids)]
                                                
    def train_dataloader(self):
        sampler = ImbalancedDatasetSampler(MelSpecDataset(data_dir = self.data_dir,
                                                          df = self.train_df,
                                                          is_train=True),
                                           MelSpecDataset.get_label)
        train_aug = aaug.Compose([
            aaug.AddGaussianNoise(p=0.3),
            aaug.AddGaussianSNR(p=0.5),
            aaug.Gain(p=0.4),
            aaug.AddBackgroundNoise(sounds_path = noise_dir, p=0.4)
            #aaug.Normalize(p=0.3)
            #aaug.Shift(p=0.2)
            #aaug.PitchShift(min_semitones=-4, max_semitones=4, p=0.3),
            #aaug.TimeStretch(p = 0.3)
       ])

        return DataLoader(
            MelSpecDataset(
                data_dir=self.data_dir,
                df=self.train_df, 
                is_train=True,
                waveform_transforms=train_aug,
            ),
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True,
            drop_last=True,
            sampler = sampler
        )

    def val_dataloader(self):
        return DataLoader(
            MelSpecDataset(
                data_dir=self.data_dir,
                df=self.val_df, 
                is_train=True
            ), 
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True,
            drop_last=False
        )

## Parameters

In [14]:
args = {
    'sample_rate': sample_rate,
    'window_size': window_size,
    'hop_size': hop_size,
    'mel_bins': mel_bins,
    'fmin': fmin, 
    'fmax': fmax,
    'classes_num': classes_num
}

r_args = {
    'sample_rate': sample_rate,
    'window_size': window_size,
    'hop_size': hop_size,
    'mel_bins': mel_bins,
    'fmin': 40, 
    'fmax': 24000,
    'classes_num': 24
}

In [15]:
weights_path = {

    "effnet": {
       "fold0": os.path.join('../input/effen-sed-rcfx-gpu/PANNsEfficientNetb4_fold0_-epoch=26.ckpt'),
        "fold1":os.path.join('../input/effen-sed-rcfx-gpu/PANNsEfficientNetb4_fold1_-epoch=32.ckpt'),
        "fold2":os.path.join('../input/effen-sed-rcfx-gpu/PANNsEfficientNetb4_fold2_-epoch=32.ckpt'),        
        "fold3":os.path.join('../input/effen-sed-rcfx-gpu/PANNsEfficientNetb4_fold3_-epoch=31.ckpt'),
        "fold4":os.path.join('../input/effen-sed-rcfx-gpu/PANNsEfficientNetb4_fold4_-epoch=26.ckpt'),
   },
}

ratio = {
   
    "fold0": 0.2,
    "fold1": 0.2,
    "fold2": 0.2,
    "fold3": 0.2,
    "fold4": 0.2,
}

In [16]:
def get_model(args: dict, weights_path: dict):
    models = {}
    device = DEVICE
    for model_key in weights_path:
        path_dict = weights_path[model_key]
        for path_key in path_dict:
            if model_key == "effnet":
                model = PANNsEfficientNet(**args)
                model = LightModel(model).to(DEVICE)
            else:
                model = PANNsResNet50Att(**args)
                model = LightModel(model).to(DEVICE)
            checkpoint = torch.load(path_dict[path_key], map_location = 'cpu')
            
            model_state_dict = {}
            
            for key in checkpoint["state_dict"]:
                if key == "n_averaged":
                    continue
                new_key = key.replace("module.", "")
                
                model_state_dict[new_key] = checkpoint["state_dict"][key]
            
            #checkpoint = torch.load(path_dict[path_key], map_location = 'cpu')["state_dict"]
            model.load_state_dict(model_state_dict, 'cpu')
            model.to(device)
            model.eval()
            models[path_key] = model
    return models

In [17]:
def read_audio_test(recording_id):
    y, sr = librosa.load(
        os.path.join(data_dir, 'test' , recording_id + '.flac'),
        sr=None, mono=True, res_type="kaiser_fast"
    )

    effective_length = sample_rate*PERIOD
    # Split for enough segments to not miss anything
    segments = len(y) / effective_length
    segments = int(np.ceil(segments))
    
    features = []
    stride = 10 * sr
    y = np.stack([y[i:i+effective_length].astype(np.float32) for i in range(0, 60*sr+stride-effective_length, stride)])    
    #for i in range(0, segments):
    #    # Last segment going from the end
    #    if (i + 1) * effective_length > len(y):
    #         y = y[len(y) - effective_length:len(y)]
    #    else:
    #         y = y[i * effective_length:(i + 1) * effective_length]#

    #    features.append(y)

    return torch.tensor(y)


In [18]:
models = get_model(args,weights_path)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b5_ns-6f26d0cf.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b5_ns-6f26d0cf.pth


### Do Inference

In [19]:
def inference(clip):
    image = clip.to(DEVICE)
    outputs = {}
    with torch.no_grad():
        for key in progress_bar(models):
            prediction = models[key](image)
            framewise_outputs = prediction["framewise_output"].detach().cpu().numpy()[0]
            framewise_outputs = np.max(framewise_outputs, axis = 0)
            outputs[key] = framewise_outputs  
    
        key = list(outputs.keys())[0]
        framewise_outputs = np.zeros_like(outputs[key], dtype=np.float32)
        for key in outputs:
            framewise_outputs += ratio[key] * outputs[key]
    
    return framewise_outputs

In [20]:
def prediction(test_df: pd.DataFrame):
    
    unique_audio_id = test_df.recording_id.unique()

    warnings.filterwarnings("ignore")
    prediction_dfs = []
    for audio_id in unique_audio_id:
        with timer(f"Loading {audio_id}"):
            clip = read_audio_test(audio_id)
        
        test_df_for_audio_id = test_df.query(
            f"recording_id == '{audio_id}'").reset_index(drop=True)
        with timer(f"Prediction on {audio_id}"):
            prediction_df = inference(clip=clip)

        prediction_dfs.append(prediction_df)
    
    sub = pd.DataFrame(prediction_dfs, columns=[f"s{i}" for i in range(24)])
    sub["recording_id"] = test_df["recording_id"].values[:len(sub)]
    sub = sub[["recording_id"] + [f"s{i}" for i in range(24)]]
    return sub


In [21]:
%%time
preds = prediction(data_test)

[Loading 000316da7] start
[Loading 000316da7] done in 0.10 s
[Prediction on 000316da7] start


[Prediction on 000316da7] done in 1.44 s
[Loading 003bc2cb2] start
[Loading 003bc2cb2] done in 0.08 s
[Prediction on 003bc2cb2] start


[Prediction on 003bc2cb2] done in 0.23 s
[Loading 0061c037e] start
[Loading 0061c037e] done in 0.08 s
[Prediction on 0061c037e] start


[Prediction on 0061c037e] done in 0.22 s
[Loading 010eb14d3] start
[Loading 010eb14d3] done in 0.08 s
[Prediction on 010eb14d3] start


[Prediction on 010eb14d3] done in 0.23 s
[Loading 011318064] start
[Loading 011318064] done in 0.07 s
[Prediction on 011318064] start


...

[Prediction on ff68f3ac3] done in 0.23 s
[Loading ff973e852] start
[Loading ff973e852] done in 0.07 s
[Prediction on ff973e852] start


[Prediction on ff973e852] done in 0.23 s
[Loading ffa5cf6d6] start
[Loading ffa5cf6d6] done in 0.08 s
[Prediction on ffa5cf6d6] start


[Prediction on ffa5cf6d6] done in 0.22 s
[Loading ffa88cbb8] start
[Loading ffa88cbb8] done in 0.09 s
[Prediction on ffa88cbb8] start


[Prediction on ffa88cbb8] done in 0.23 s
[Loading ffda5d7b3] start
[Loading ffda5d7b3] done in 0.09 s
[Prediction on ffda5d7b3] start


[Prediction on ffda5d7b3] done in 0.22 s
CPU times: user 9min 16s, sys: 11.4 s, total: 9min 27s
Wall time: 10min 46s


In [22]:
preds

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,...,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,0.051132,0.003673,0.002533,0.564120,0.005593,0.162241,0.007614,0.009309,0.003524,...,0.018238,0.006160,0.001703,0.006992,0.490315,0.003486,0.009336,0.003703,0.007923,0.009448
1,003bc2cb2,0.000049,0.000961,0.000038,0.002776,0.000048,0.000065,0.000033,0.000124,0.000031,...,0.000028,0.000049,0.998118,0.000527,0.000054,0.000212,0.000647,0.000089,0.000133,0.000144
2,0061c037e,0.001358,0.001389,0.001528,0.008032,0.012200,0.067828,0.003681,0.047022,0.006854,...,0.000954,0.003584,0.002804,0.494765,0.001033,0.005439,0.270729,0.000764,0.019611,0.050941
3,010eb14d3,0.956815,0.003021,0.000216,0.005119,0.001105,0.000747,0.000241,0.000259,0.081089,...,0.000173,0.000718,0.000315,0.000110,0.031945,0.000641,0.000339,0.007976,0.002634,0.004380
4,011318064,0.000366,0.004243,0.000881,0.023265,0.000871,0.002831,0.007446,0.007755,0.000693,...,0.954930,0.399227,0.000507,0.001458,0.002658,0.000391,0.000922,0.013059,0.001114,0.001331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,ff68f3ac3,0.000410,0.000672,0.000644,0.007561,0.001238,0.130417,0.004577,0.029106,0.002346,...,0.000918,0.729743,0.000440,0.002400,0.000633,0.001687,0.006903,0.000294,0.005340,0.611383
1988,ff973e852,0.001127,0.002133,0.001626,0.060672,0.002621,0.010249,0.004023,0.477728,0.000926,...,0.019473,0.362571,0.007614,0.094789,0.001119,0.011513,0.069894,0.000788,0.042433,0.031494
1989,ffa5cf6d6,0.002459,0.023025,0.098419,0.181240,0.003133,0.055344,0.003453,0.023698,0.000587,...,0.004369,0.005923,0.004920,0.862236,0.006558,0.009694,0.012466,0.001005,0.012835,0.043544
1990,ffa88cbb8,0.011296,0.009154,0.005150,0.739462,0.004962,0.031174,0.002520,0.108158,0.002167,...,0.004786,0.008423,0.022654,0.005802,0.005489,0.001633,0.005705,0.003397,0.007266,0.208886


In [23]:
preds.to_csv(OUTPUT_DIR + "submission.csv", index=False)